In [1]:
import numpy as np
import pyopencl as cl

In [2]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [3]:
a = np.array(range(50), np.int32)
b = 100 + np.array(range(50), np.int32)

c = np.empty_like(a)

In [4]:
mf = cl.mem_flags
a_buf = cl.Buffer(ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=a)
b_buf = cl.Buffer(ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=b)

c_buf = cl.Buffer(ctx, cl.mem_flags.WRITE_ONLY, a.nbytes)

In [5]:
kernel = """
__kernel void foo(__global int* a, __global int* b, __global int* c) 
{
    int i = get_global_id(0);
    c[i] = a[i] * b[i];
}
"""

In [6]:
program = cl.Program(ctx, kernel).build()
launch = program.foo(queue, a.shape, None,
                        a_buf, b_buf, c_buf)
launch.wait()
cl.enqueue_copy(queue, c, c_buf).wait()

In [7]:
assert np.all(c == (a * b))
print(c)

[   0  101  204  309  416  525  636  749  864  981 1100 1221 1344 1469 1596
 1725 1856 1989 2124 2261 2400 2541 2684 2829 2976 3125 3276 3429 3584 3741
 3900 4061 4224 4389 4556 4725 4896 5069 5244 5421 5600 5781 5964 6149 6336
 6525 6716 6909 7104 7301]
